In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
ruta_lectura = "procien/"
ruta_final = "procien/finales/"

## Investigadores asociados

In [3]:
anios = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
xls_base = pd.read_excel(f"{ruta_lectura}PROCIEN_Investigadores_julio.2024.xlsx", sheet_name=anios)

In [4]:
lista_dataframes_personas = []
for n in anios:
    print(f"fecha {n}")
    tmp_xls = xls_base[n][['Código', 'Nombre IP', 'Equipo chileno', 'Instituciones', 'Equipo extranjero', 'Institución', 'País', 'Género']]#.dropna()
    
    lista = []
    ##lista de paises
    for pais, pers1, pers2 in zip(tmp_xls['País'], tmp_xls['Equipo chileno'], tmp_xls['Equipo extranjero']):
        if pd.isna(pais):
            if not pd.isna( pers1 ) or not pd.isna( pers2 ):
                lista.append('')
            else:
                pass
        else:
            lista.append(pais)

    lista1 = []
    ##Unir equipo chileno con extranjero
    for p1, p2 in zip( tmp_xls['Equipo chileno'], tmp_xls['Equipo extranjero'] ):
        #print(p1,p2)
        if pd.isna(p1) and pd.isna(p2):
            #print("aca")
            pass
        elif pd.isna(p1):
            lista1.append( p2 )
        elif pd.isna(p2):
            lista1.append( p1 )
        else:
            print(p1,p2)

    lista2 = []
    ##Unir instituciones
    for i, (perso, lugar1, lugar2) in enumerate( zip(lista1, tmp_xls['Instituciones'], tmp_xls['Institución'] ) ):
        if pd.isna(lugar1) and pd.isna(lugar2):
            if not pd.isna(perso):
                lista2.append( '' )
            else:
                pass
        elif pd.isna(lugar1):
            lista2.append( lugar2 )
        elif pd.isna(lugar2):
            lista2.append( lugar1 )
        else:
            lista2.append( '' )

    lista3 = []
    ##lista con el genero
    for g in tmp_xls['Género']:
        if pd.isna(g):
            pass
        else:
            lista3.append(g)

    lista4 = []
    ##lista con los codigos
    code_base = ''
    for code, gen in zip(tmp_xls['Código'], tmp_xls['Género']):
        if code_base != code and not pd.isna(code):
            code_base = code
        
        if not pd.isna(gen):
            lista4.append(code_base) 

    lista5 = []
    for _ in range( len(lista4) ):
        lista5.append( n )

    new_dataframe = [lista, lista1, lista2, lista3, lista4, lista5]
    lista_dataframes_personas.append( pd.DataFrame(np.array(new_dataframe).T, columns=['pais', 'nombre', 'afiliacion', 'genero', 'codigo', 'anio']))

fecha 2014
fecha 2015
fecha 2016
fecha 2017
fecha 2018
fecha 2019
fecha 2020
fecha 2021
fecha 2022
fecha 2023
fecha 2024


In [ ]:
full_dataframe = pd.concat( lista_dataframes_personas )
full_dataframe.to_excel(f"{ruta_final}asociados-procien.xlsx")

## Investigadores principales

In [24]:
def agrupacion_general(codigo):
    final = ''
    if codigo in ["MT", "MG", "DT", "DG", "RG", "AMP", "RT", "AM"]:
        final = "INACH"
    elif codigo in ["FP", "FI", "FR", "PR"]:
        final = "FONDECYT"
    elif codigo in ["FOVI", "FV"]:
        final = "FOMENTO"
    elif codigo in ["INT", "IN"]:
        final = "INTER"
    elif codigo in ["FE"]:
        final = "FONDEQUIP"
    elif codigo in ["NE"]:
        final = "NODO"
    elif codigo in ["AN"]:
        final = "ANILLO"
    elif codigo in ["MI", "IM"]:
        final = "MILENIO"
    elif codigo in ["FD"]:
        final = "FONDAP"
    return final


def macrozona_anid(string):
    if string in ['Antofagasta']:
        return "norte"
    elif string in ['Valparaiso', 'Valparaíso', 'Viña del Mar', 'Coquimbo', 'La Serena']:
        return "centro"
    elif string in ['Concepción', 'Talcahuano', 'Talca', 'Chillán', 'Los Angeles']:
        return "centro sur"
    elif string in ['Temuco', 'Temúco', 'Valdivia', 'Pto. Montt', 'Puerto Montt', 'Pto Montt']:
        return "sur"
    elif string in ['Punta Arenas', 'Pto. Williams', 'Puerto Williams']:
        return "austral"
    elif string in ['Santiago']:
        return "metropolitana"
    else:
        return ''

In [25]:
anios = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
xls_base = pd.read_excel(f"{ruta_lectura}PROCIEN_Investigadores_julio.2024.xlsx", 
    sheet_name=anios)

In [26]:
lista_xls_ip = []
for n in anios:
    try:
        tmp_xls = xls_base[n][['Código', 'Título corto', 'Nombre IP', 'Ciudad', 'Instituciones', 'Género']].to_numpy()
    except:
        tmp_xls = xls_base[n][['Código', 'Nombre corto', 'Nombre IP', 'Ciudad', 'Instituciones', 'Género']].to_numpy()

    for code, titulo, name, ciudad, inst, genero in tmp_xls:
        if pd.isna(name):
            pass
        else:
            if " - " in name:
                name = name.split("-")
                name = [n.strip() for n in name]
            
            elif " / " in name:
                name = name.split("/")
                name = [n.strip() for n in name]
            else:
                pass
            
            #print( n, code, titulo, name, ciudad, inst, genero )
            codigo = code.split('_')[0].strip().upper()
            codigo_general = agrupacion_general(codigo)
            

            if pd.isna(titulo):
                titulo = ''

            if type(name) == list:
                for no in name:
                    lista_xls_ip.append( [code, codigo, codigo_general, n, titulo.lower(), 
                        no.lower(), macrozona_anid(ciudad), inst, genero] )
            else:
                lista_xls_ip.append( [code, codigo, codigo_general, n, titulo.lower(), 
                        name.lower(), macrozona_anid(ciudad), inst, genero] )

In [27]:
aux_dataframe = pd.DataFrame(lista_xls_ip, columns=['codigo', 'tipo proyecto', 'agrupacion proyecto', 'anio', 'titulo', 'nombre ip', 'macrozona', 'institucion', 'genero'])
aux_dataframe.to_excel(f"{ruta_final}investigadores-principales-procien.xlsx")